In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 64 output channels, 1x6  convolution
        # kernel
        self.local_conv = nn.Conv2d(1, 64, (1,6))
        
        # 64 input channels (check this?), feature maps from local convolution,
        # 128 output channels, 20x2 kernel (check this?)
        self.global_conv=nn.Conv2d(64,128,(20,2))
        
        #LSTM layer,48 cells each,  how to set 0.25 dropout rate ???
        self.dec=nn.LSTM(128,48,2,dropout=0.25)
        
        # Size of output of LSTM, for now use # of hiden state features
        self.denseFF=nn.Linear(48,7)
        self.sm=nn.Softmax()
        
    def forward(self, x,hidden):
        # Apply ReLu units to the results of convolution, local convoltion layer
        x=F.relu(self.local_conv(x))
        x=nn.MaxPool1d(4)(x)
        
        #Global convolution layer
        x=F.relu(self.global_conv(x))
        x=nn.MaxPool1d(2)(x)
        
        out,hidden=self.dec(x,hidden)
        
        # Feed output through dense dense/feedforward layer with softmax activation units to 
        # classify the input onto one of the 7 emotion categories.
        out=self.sm(self.denseFF(out))
        return out
        



net = Net()
print(net)

Net(
  (local_conv): Conv2d(1, 64, kernel_size=(1, 6), stride=(1, 1))
  (global_conv): Conv2d(64, 128, kernel_size=(20, 2), stride=(1, 1))
  (dec): LSTM(128, 48, num_layers=2, dropout=0.25)
  (denseFF): Linear(in_features=48, out_features=7, bias=True)
  (sm): Softmax()
)


In [ ]:
# training 

